# Exploratory Data Analysis - King County House Prices

### Description about the Project

In this notebook I will take a look at a dataset that includes information about the King County housing market. Further I will try to assist Mr. Henson (imaginary Person) and his family to find a fitting home and to provide insights that will help them optimize their search strategy. 

I am provided with the following information about Mr. Henson:
- 5 kids
- no money
- wants nice (social) neighborhood
<br>

At first I will import all required packages for the project. For visualization I will use the library plotly.

In [ ]:
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import plotly.express as px
import geojson

pd.set_option('display.float_format', lambda x: '%.3f' % x)

<br>
Here I will read the .csv file that includes the data I will investigate. I also change the date column into date format as well as add another column that only includes the month.
<br>

In [ ]:
# Read the Dataframe of King County house prices
houses = pd.read_csv("data/King_County_House_prices_dataset.csv")

# Convert the date column into type date
houses['date']= pd.to_datetime(houses['date'])

# Add one column that only shows the month
houses["month"] = pd.DatetimeIndex(houses['date']).month

<br>
I will now have a look and go through the data to think about possible approaches to find interesting insights. As the meaning of the column names are not immediately clear, here is a quick overview of the meanings:

- id - unique identified for a house
- dateDate - house was sold
- pricePrice - is prediction target
- bedroomsNumber - # of bedrooms
- bathroomsNumber - # of bathrooms
- sqft_livingsquare - footage of the home
- sqft_lotsquare - footage of the lot
- floorsTotal - floors (levels) in house
- waterfront - House which has a view to a waterfront
- view - Has been viewed
- condition - How good the condition is ( Overall )
- grade - overall grade given to the housing unit, based on King County grading system
- sqft_above - square footage of house apart from basement
- sqft_basement - square footage of the basement
- yr_built - Built Year
- yr_renovated - Year when house was renovated
- zipcode - zip
- lat - Latitude coordinate
- long - Longitude coordinate
- sqft_living15 - The square footage of interior housing living space for the nearest 15 neighbors
- sqft_lot15 - The square footage of the land lots of the nearest 15 neighbors
<br>

After looking through the dataset and knowing that the Henson's are 7 people, I want to understand how many houses fitting the bedroom requirements are out there. I assume that at least 4 bedrooms are needed. 
I will now plot the available houses by the amount of bedrooms.

In [ ]:
# There are some outliers, such as houses with 33 bedrooms which I will drop from the list. 
# For the first plot I will present my data from 0 - 10 bedrooms
houses_by_bedroom_2 = houses.drop(houses[houses["bedrooms"] > 10].index)

# Using px.histogram with "bedrooms" on the x-axis to automatically let plotly count the amounts on the y-axis
fig = px.histogram(houses_by_bedroom_2, x="bedrooms", text_auto=True, title="<b>Amount of available houses by amount of bedrooms")
fig.update_layout(
        xaxis=dict(
                title_text="<b>Bedrooms",
                tickvals=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                titlefont=dict(size=14)),
        yaxis=dict(
                title_text="<b>Count",
                titlefont=dict(size=14)),
        yaxis_range=[0,12000],
        bargap=0.1, 
        width=1000, height=400)
fig.update_traces(
        textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.show()

<img src='images/1.png'>

The histogram shows us, that there is a very limited number for houses with more then 6 bedrooms. I recommend to look for houses starting from 4 bedrooms. As Mr. Henson has no money there is no room for any luxury and it may be needed to take houses with smaller living areas as well as bedrooms. 
<br>

Next I want to understand, if the price is reflecting the amount of bedrooms properly. With other words: Do you get what you pay for in terms of bedrooms?

In [ ]:
# Get the average price grouped by the amount of bedrooms
houses_bedrooms = houses.groupby("bedrooms")["price"].mean().reset_index()

# Count the appearances of each bedroom type
houses_bedrooms_count = houses.groupby("bedrooms")["id"].count().reset_index()

# Combine both lists together with pd.conact
houses_bedrooms_incl_count = pd.concat([houses_bedrooms.set_index("bedrooms"), 
        houses_bedrooms_count.set_index("bedrooms")], axis=1).reset_index()

# In order to show only relevant numbers, I remove all rows that have less then 100 observations 
houses_bedrooms_incl_count = houses_bedrooms_incl_count.drop(houses_bedrooms_incl_count[houses_bedrooms_incl_count["id"] < 100].index)

# Plot results with px.scatter and add a trendline to visualize correlation
fig_2 = px.scatter(houses_bedrooms_incl_count, x="bedrooms", y="price", text="id", trendline="ols", 
        title="<b>Price per amount of bedrooms")
fig_2.update_layout(
        xaxis=dict(
                title_text="<b>Bedrooms",
                titlefont=dict(size=14)),
        yaxis=dict(
                title_text="<b>Price",
                titlefont=dict(size=14)),
                width=1000, height=500)
fig_2.update_traces(
        marker=dict(
                size=16, line=dict(width=1, color='black')), 
        textposition="top left")
fig_2.show()

<img src='images/2.png'>

The answer seems to be "yes". We can see that there is a strong correlation between bedrooms and house price. Therefore Mr. Hansen does not need to look out for a specific amount of bedrooms.
Note: The reason that there are no houses in the graph with 7 bedrooms or more is that all these houses have less then 100 observations and were ignored.
<br>

Now that we know about bedrooms I want to understand a very similar fact with comparing the living area to the price. 

In [ ]:
# Get the average living area grouped by the amount of bedrooms
houses_living = houses.groupby("bedrooms")["sqft_living"].mean().reset_index()

# Count the appearances of each bedroom type
houses_living_count = houses.groupby("bedrooms")["id"].count().reset_index()

# Combine both lists together with pd.conact
houses_living_incl_count = pd.concat([houses_living.set_index("bedrooms"), 
        houses_living_count.set_index("bedrooms")], axis=1).reset_index()

# In order to show only relevant numbers, I remove all rows that have less then 100 observations 
houses_living_incl_count = houses_living_incl_count.drop(houses_living_incl_count[houses_living_incl_count["id"] < 100].index)

# Plot results with px.scatter and add a trendline to visualize correlation
fig_3 = px.scatter(houses_living_incl_count, x="bedrooms", y="sqft_living",
        text="id", title="<b>Average living space by amount of Bedrooms", trendline="ols")
fig_3.update_layout(
        xaxis=dict(
        title_text="<b>Bedrooms",
        titlefont=dict(size=14)),
        yaxis=dict(
        title_text="<b>Living area",
        titlefont=dict(size=14)),
        width=1000, height=500)
fig_3.update_traces(
        marker=dict(size=16, line=dict(width=1, color='black')), 
        textposition="bottom left")
fig_3.show()

<img src='images/3.png'>

Similar to the graph before, there is also a strong correlation between bedrooms and living area. Nevertheless we can see that 4 and 5 bedroom houses tend to have a slighlty better living area relation then 6 bedroom houses. This is a helpful information because there are not many 6 bedrooms houses available and its good to know that 4 and 5 bedroom houses tend to have a better relation - even though its not dramatic.
<br>

Next I am focussing on the time and want to understand, if I can find a recommendation on when to buy. I plan on calculation the average price per month for all houses with more then 4 bedrooms.

In [ ]:
# Simple query to get all houses with 4 or more bedrooms
houses_4_bedrooms_plus = houses.query("bedrooms >= 4")

# Next I group the list by month and get the average price for each month
houses_avg_per_month = houses_4_bedrooms_plus.groupby("month")["price"].mean().reset_index()

# In this step the count of each house sold will be calculated
houses_sold_per_month = houses_4_bedrooms_plus.groupby("month")["id"].count().reset_index()

# Combining both lists with pd.concat so we have avg price and counts per month
houses_avg_price_and_count_per_month = pd.concat([houses_sold_per_month.set_index("month"), 
        houses_avg_per_month.set_index("month")], axis=1).reset_index()

# Show a scatter plot - Sizes of dots show the counts and the color indicates about the avg prices.
fig_4 = px.scatter(houses_avg_price_and_count_per_month, x="month", y="price", color="id", size="id", 
        title="<b>4+ bedrooms: Average house prices per month <br><sup>Color indicates amount of houses sold</sup>")
fig_4.update_layout(
        xaxis=dict(
                title_text="<b>Month",
                tickmode = 'array',
                tickvals = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
                ticktext = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"],
                titlefont=dict(size=14)),
        yaxis=dict(
                title_text="<b>Price",
                titlefont=dict(size=14)),
                width=1000, height=500,
                yaxis_range=[550000,800000], 
        coloraxis_colorbar=dict(
                title="<b>Amount <br><sup>Houses sold</sup>"))
fig_4.show()

<img src='images/4.png'>

With this nice visualization we can see that in February the avg price for houses with 4+ bedrooms was the lowest and in May it was the highest. We still have to take the count of houses sold in consideration and understand, that in February fewer houses have been sold than in May. Nevertheless this graph is a nice indicator that chances may be higher to find a good offer in the lower avg months.
<br>

Last I want to find out about the distribution of prices and amounts of houses sold geographically. As the dataset includes the zipcodes I plan to visualize the zipcodes area. For this I have included a zipcode geojson file that will be needed to match the locations. As in the above plots I will still only focus on houses with 4 or more bedrooms.

In [ ]:
# Preparing DF for geo plot. Getting avg prices per zipcode as well as the counts of houses sold per zipcode. 
zip_codes_price_avg = houses_4_bedrooms_plus.groupby("zipcode")["price"].mean().reset_index()
zip_codes_counts = houses_4_bedrooms_plus.groupby("zipcode")["id"].count().reset_index()

# Join the two tables with concat
zip_codes = pd.concat([zip_codes_price_avg.set_index("zipcode"), zip_codes_counts.set_index("zipcode")], axis=1).reset_index()

# Load geo json file for King County area
with open ("data/Zip_Codes.geojson") as f:
    gj = geojson.load(f)

# adding the zipcodes as "id" so plotly can access the data properly. Geo json data is then stored in a new variable called "gj"
for i in range(len(gj["features"])):
    gj["features"][i]["id"] = gj["features"][i]["properties"]["ZIPCODE"]

In [ ]:
# Plotting the average prices per Zipcode using "px.choropleth_mapbox". Locations ("zipcode") are matched with geojson file ("gj"). 
# Color is added to visualize the avg prices and the map is centered at Seattles lat and lon coordinates.

fig_geo = px.choropleth_mapbox(zip_codes, geojson=gj, locations="zipcode", mapbox_style="carto-positron", color="price", 
        center={'lat': 47.453348, 'lon': -122.036926}, color_continuous_scale="deep")
fig_geo.update_layout(
        coloraxis_colorbar=dict(
                title="<b>Avg Price <br><sup>per Zipcode</sup>",
                lenmode="pixels", len=300),
        width=1000, height=500,
        margin={'r':0,'t':0,'l':0,'b':0})
fig_geo.show()

<img src='images/5.png'>

We can see that the average prices are highest in the center and the more you move away from it the lower the prices get. This is not a huge surprise but still good to know. A very helpful information though is that the prices in the south tend to be lower then in the north.
<br>

Next I want to show not the average prices but the average amount of houses available per zipcode.

In [ ]:
# Everything is the same as above just the color was changed to "id" in order to represent the amount of houses available.

fig_geo_2 = px.choropleth_mapbox(zip_codes, geojson=gj, locations="zipcode", mapbox_style="carto-positron",
        color="id", center={'lat': 47.453348, 'lon': -122.036926}, color_continuous_scale="deep")
fig_geo_2.update_layout(margin={'r':0,'t':0,'l':0,'b':0})
fig_geo_2.update_layout(
        coloraxis_colorbar=dict(
                title="<b>Amount <br><sup>Houses sold</sup>",
                lenmode="pixels", len=300),
        width=1000, height=500)
fig_geo_2.show()

<img src='images/6.png'>

We now have a nice overview on how many houses are available in which area.
<br>

Finally I want to somehow combine the two geo plots to have a nice overview where cheap houses are available and a lot of houses are offered. For this I will only display zipcodes where the avg price is in the lower half of all avg prices and zipcodes where the avg amount of houses offered is in the higher half of all amounts of houses offered. This should give us a decent idea on where the most lower prices houses are available.

In [ ]:
# Getting the lower half of all avg prices and lower half of all avg counts
prices_mean_half = houses_4_bedrooms_plus.price.mean() / 2
count_mean_half = zip_codes_counts.id.mean() / 2

# Adjust zip_code df with the newly created filters by doing a simple query
zip_codes_filtered = zip_codes.query("price < @prices_mean_half and id > @count_mean_half")

fig_geo_3 = px.choropleth_mapbox(zip_codes_filtered, geojson=gj, locations="zipcode", mapbox_style="carto-positron",
                center={'lat': 47.353348, 'lon': -122.436926})
fig_geo_3.update_layout(
        width=600, height=400,
        margin={'r':0,'t':0,'l':0,'b':0})
fig_geo_3.show()

<img src='images/7.png'>

With this final geo plot we have a very nice breakdown on where we find a decent amount of 4+ bedroom houses at low prices. In these areas we should have the best chances i.e. highest probability to find a matching house for Mr. Henson and his family.
<br>

### Recommendation and conclusion of the project:
<br>

- Mr. Henson and his family should start looking for houses with 4 bedrooms or more, mainly because houses with more bedrooms are very limited and Mr. Henson has no money
- Looking at the data for bedroom/price and living area/price the relation seems to be quite fair in this area. Due to slight differences the chances to get most for your money in terms of bedroom/living area are a bit higher with 4 and 5 bedroom houses
- In terms of timing it is definitely recommended to check out the market in February as we have seen that avg prices are the lowest.
- Regarding the location we created a very nice plot that shows where many houses at low prices are available. In these areas chances for a good offer are the highest.